In [ ]:
def factorial_mod(k, n):
    """função que calcula k! mod n"""
    Zn = IntegerModRing(n)
    factorial = Zn(1)
    
    for j in range(1, k+1):
        factorial = factorial * j
    
    return ZZ(factorial)


def teste_Wilson(n):
    """Testa a primalidade de um natural usando o Teorema de Wilson"""
    return mod(factorial_mod(n-1, n), n) == mod(-1, n)

In [ ]:
def teste_Fermat(n, base = 2):
    if gcd(base, n) != 1:
        return False
    
    return power_mod(base, n-1, n) == 1
# o power_mod faz base^(n-1) % n

teste_Fermat(103)

### Teste de Miller

In [16]:
def decomp(n):
    s = 0
    t = n-1
    while t % 2 == 0:
        t = t // 2
        s+=1
    return s, t
decomp(n)

(5, 3)

In [19]:
n = 97
def Miller(n, b=2):
    Zn = IntegerModRing(n)
    b = Zn(b)
    s, t = decomp(n)
    if b^t == 1:
        return True
    j = 0
    while b^(2^j *t) != Zn(-1) and j<= s-1:
        j = j+1
    return b^(2^j *t) == Zn(-1)
Miller(13)

True

## RSA 

#### Fatorização usando o teorema de fatorização de Fermat

In [5]:
def crack_when_pq_close(n):
    """When p and q are Close (uses Fermat Factorization Method) p.62"""
    t = Integer(ceil(sqrt(n)))
    while True:
        k = t^2 - n
        if k > 0:
            s = Integer(int(round(sqrt(t^2 - n))))
            if s^2 + n == t^2:
                return t+s, t-s

        t += 1
crack_when_pq_close(1025)

(41, 25)

In [6]:
def ρ_pollard(n):
    """Cormen, Thomas H.; Leiserson, Charles E.; Rivest, Ronald L. & Stein, Clifford (2009). "Section 31.9: Integer factorization". Introduction to Algorithms (third ed.). Cambridge, MA: MIT Press. pp. 975–980. ISBN 978-0-262-03384-8. (this section discusses only Pollard's rho algorithm)."""
    x = 2
    y = 2
    d = 1
    g = lambda x: mod(x**2+1,n)
    while d == 1:
        x = g(x)
        y = g(g(y))
        d = gcd(x - y, n)
    if d == n:
        return None
    else:
        return d
    
ρ_pollard(1024)

64

In [10]:
def ρ_menos_um_pollard(N, B=10^5, stop=10):
    """Pollard’s (p − 1)-Method"""
    m = prod([p^int(math.log(B)/math.log(p)) for p in prime_range(B+1)])
    for a in [2..stop]:
        x = (Mod(a,N)^m - 1).lift()
        if x == 0: continue
        g = gcd(x, N)
        if g != 1 or g != N: return g
    return 1

ρ_menos_um_pollard(1278)

2

#### Fatorizar n tendo φ(n)

In [ ]:
def crack_rsa(n, phi_n):
    """Factoring n Given φ(n)"""
    R.<x> = PolynomialRing(QQ)
    f = x^2 - (n+1 - phi_n)*x + n
    return [b for b, _ in f.roots()]



In [10]:
def findOrders(n):
    Zn = IntegerModRing(n)
    srr = [a for a in Zn if gcd(a,n)==1]
    return pretty_print([(r,Zn(r).multiplicative_order()) for r in srr])

def findRoots(n):
    phi = euler_phi(n)
    Zn = IntegerModRing(n)
    srr = [a for a in Zn if gcd(a,n)==1]
    return [r for r in srr if Zn(r).multiplicative_order()== phi]

def findSrr(n):
    phi = euler_phi(n)
    Zn = IntegerModRing(n)
    return [a for a in Zn if gcd(a,n)==1] 

def findSrr2(n):
    return IntegerModRing(n).list_of_elements_of_multiplicative_group()

### RSA Generation, encryption and decryption
PubKey = (e, n) ||
PrivKey = (d, n)

Encryption -  x^e mod N ||||
Decryption - (x^e)^d mod N


In [3]:
def RSA_gen(nbits):
    p = random_prime(2^(nbits//2+1), lbound=2^(nbits // 2))
    q = random_prime(2^(nbits//2), lbound=2^(nbits//2-1))
    n = p*q
    m = (p-1)*(q-1)
    e = randint(2, m-1)
    
    while gcd(e,m) != 1:
        e = randint(2, m-1)
        
    d = power_mod(e, -1, m)
    PubKey = (n, e)
    PrivKey = (n, d)
    
    return PubKey, PrivKey

def RSA_encrypt(PubKey, mens):
    n, e = PubKey
    return power_mod(mens, e, n)

def RSA_decrypt(PrivKey, cypherText):
    n, d = PrivKey
    return power_mod(cypherText, d, n)

RSA_encrypt((630148901, 41859065), 1234)

474676446

In [2]:
def rsa_e(bits,e):
    # only prove correctness up to 1024 bits
    proof = (bits <= 1024)
    while True:
        p = next_prime(ZZ.random_element(2**(bits//2+1)),
                proof=proof)
        q = next_prime(ZZ.random_element(2**(bits//2+1)),
                proof=proof)
        n = p*q
        phi_n = (p-1)*(q-1)
        if gcd(e,phi_n) == 1: break
    d = lift(Mod(e,phi_n)^(-1))
    return d, n, e

In [ ]:
n=1431702961131339621602945101050088245802771644687686106472404992618656781417216251296575852739319584928520070989102317406096771341195807540255351846662720304620283140579998725212022478694288443
phi_n = 1431702961131339621602945101050088245793639679386713169093313712250573769060652354811397118799813658291947433707988011985679885048571032964071342275971679191493672348651555118274559858896490208
